<a href="https://colab.research.google.com/github/jai2shan/TSAI-EVA40-Assignments/blob/master/Session%204/EVA4_Session_4%20including%20Batch%20Norm%20Step%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Batch Normalization - Trial
 Before Applying    
Current Test Accuracy - Accuracy: 9849/10000 (98%)     
No of Parameters - 10,790   

After applying    
New Test Accuracy - 99.2    
New Parameters - 10,790
 

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels = 1, out_channels = 10, kernel_size = (3,3), padding=0,bias = False), #input -28 Output - 26, RF - 3
                                   nn.ReLU(),
                                   nn.BatchNorm2d(10)) 
        
        self.conv2 = nn.Sequential(nn.Conv2d(in_channels = 10, out_channels = 10, kernel_size = (3,3), padding=0,bias = False), #input - 26, Output - 24, RF- 5
                                   nn.ReLU(),
                                   nn.BatchNorm2d(10))
        
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels = 10, out_channels = 20, kernel_size = (3,3), padding=0,bias = False), #input - 24, Output - 22, RF- 5
                                   nn.ReLU(),
                                   nn.BatchNorm2d(20))
                
        self.pool1 = nn.Sequential(nn.MaxPool2d(2, 2)) #input 22, Output - 11

        self.conv4 = nn.Sequential(nn.Conv2d(in_channels = 20, out_channels = 10, kernel_size = (1,1), padding=0,bias = False), #input 11, Output - 11
                                   nn.ReLU(),
                                   nn.BatchNorm2d(10))
        
        self.conv5 = nn.Sequential(nn.Conv2d(in_channels = 10, out_channels = 10, kernel_size = (3,3), padding=0,bias = False), #input 11, Output - 9
                                   nn.ReLU(),
                                   nn.BatchNorm2d(10))
        
        self.conv6 = nn.Sequential(nn.Conv2d(in_channels = 10, out_channels = 20, kernel_size = (3,3), padding=0,bias = False), #input 9, Output - 7
                                   nn.ReLU(),
                                   nn.BatchNorm2d(20))
        
        self.conv7 = nn.Sequential(nn.Conv2d(in_channels = 20, out_channels = 10, kernel_size = (1,1), padding=0,bias = False), #input 7, Output - 7
                                   nn.ReLU(),
                                   nn.BatchNorm2d(10))
        
        self.conv8 = nn.Sequential(nn.Conv2d(in_channels = 10, out_channels = 10, kernel_size = (7,7), padding=0,bias = False)) #input 7, Output - 11
                            
    def forward(self, x): 
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x) 
        x = self.pool1(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [9]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
            Conv2d-4           [-1, 10, 24, 24]             900
              ReLU-5           [-1, 10, 24, 24]               0
       BatchNorm2d-6           [-1, 10, 24, 24]              20
            Conv2d-7           [-1, 20, 22, 22]           1,800
              ReLU-8           [-1, 20, 22, 22]               0
       BatchNorm2d-9           [-1, 20, 22, 22]              40
        MaxPool2d-10           [-1, 20, 11, 11]               0
           Conv2d-11           [-1, 10, 11, 11]             200
             ReLU-12           [-1, 10, 11, 11]               0
      BatchNorm2d-13           [-1, 10, 11, 11]              20
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0576, Accuracy: 9808/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0361, Accuracy: 9885/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0283, Accuracy: 9920/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0264, Accuracy: 9925/10000 (99%)



loss=0.0008849054574966431 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 44.52it/s]



Test set: Average loss: 0.0265, Accuracy: 9916/10000 (99%)

